**Named Entity Recognizer**

Instructions:

You can find a corpus, possibly tagged with Arabic Named Entities or create a dataset that contains a few sentences, manually label the named entities.

1- Load your corpus to a Data Frame

2- Apply the necessary preprocessing steps using regEx, tokenization and/or lemmatization, in case you created your own corpus.

3- Develop your own NER. It could be rule-based, or you could build a predictive machine learning model for the task using the annotated corpus.

4- Evaluate the results using accuracy, recall, etc. and analyze the results.

***Step 0: Install Farasa***

In [1]:
#install Java and Farasapy
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install farasapy


In [2]:
import pandas as pd

from farasa.segmenter import FarasaSegmenter
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.stemmer import FarasaStemmer
#initialize Farasa Segmenter and NER
farasa_segmenter = FarasaSegmenter(interactive=True)
farasa_ner = FarasaNamedEntityRecognizer(interactive=True)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [03:06<00:00, 1.29MiB/s]


[2024-09-19 13:52:41,701 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2024-09-19 13:52:47,924 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


Install Farasa in Step 0 to process Arabic text.
To achieve advanced tokenization and lemmatization in Arabic, install the Farasa library.

***Step 1: Create the dataset with sentences and manually labeled entities***

In [3]:
data = {
    'sentence': [
        'زار البابا فرنسيس مدينة أبوظبي لتعزيز الحوار بين الأديان.',
        'أعلن نادي برشلونة عن انتقال اللاعب البرتغالي كريستيانو رونالدو إلى صفوفه.',
        'افتتحت شركة سامسونج معرضاً جديداً في مدينة دبي.',
        'زار الرئيس الفرنسي إيمانويل ماكرون لبنان بعد انفجار مرفأ بيروت.',
        'ألقى الشاعر نزار قباني قصيدة في مهرجان قرطاج الدولي.',
        'تعاقدت وزارة الصحة المصرية مع منظمة اليونيسف لتحسين خدمات الصحة العامة.',
        'تم اكتشاف تمثال جديد في منطقة الأهرامات في مصر.',
        'عقدت قمة المناخ في العاصمة الدنماركية كوبنهاغن بمشاركة قادة العالم.',
        'أعلنت جامعة القاهرة عن مؤتمرها السنوي حول الذكاء الاصطناعي.',
        'وقعت دولة الإمارات اتفاقية تعاون اقتصادي مع الهند.'
    ],
    'entities': [
        [('البابا فرنسيس', 'Person'), ('مدينة أبوظبي', 'Location')],
        [('نادي برشلونة', 'Organization'), ('اللاعب البرتغالي', 'Position'), ('كريستيانو رونالدو', 'Person')],
        [('شركة سامسونج', 'Organization'), ('مدينة دبي', 'Location')],
        [('الرئيس الفرنسي', 'Position'), ('إيمانويل ماكرون', 'Person'), ('لبنان', 'Location'), ('مرفأ بيروت', 'Location')],
        [('الشاعر نزار قباني', 'Person'), ('مهرجان قرطاج الدولي', 'Event')],
        [('وزارة الصحة المصرية', 'Organization'), ('منظمة اليونيسف', 'Organization')],
        [('منطقة الأهرامات', 'Location'), ('مصر', 'Location')],
        [('قمة المناخ', 'Event'), ('العاصمة الدنماركية كوبنهاغن', 'Location')],
        [('جامعة القاهرة', 'Organization'), ('مؤتمرها السنوي', 'Event')],
        [('دولة الإمارات', 'Location'), ('الهند', 'Location')]
    ]
}

In [4]:
#create a DataFrame from the corpus
df = pd.DataFrame(data)

In [5]:
#save data into a csv file
df.to_csv('arabic_ner_corpus.csv', index=False, encoding='utf-8-sig')

Step 1 involved the creation of an Arabic sentence dataset with manual entity annotations. For processing, this data is put into a pandas DataFrame.

***Step 2: Apply Preprocessing Steps***

In [6]:
import re
#initialize the Farasa Stemmer
stemmer = FarasaStemmer()

def preprocess_text(text):
    #step A: RegEx Cleaning - Remove any non-Arabic letters, numbers, and special characters
    text = re.sub(r'[^ا-ي\s]', '', text)

    #step B: Tokenization - Split text into words
    tokens = text.split()

    #step C: Stemming - Apply Farasa Stemmer to each token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    #return the stemmed text
    return " ".join(stemmed_tokens)

#apply the preprocessing function to the 'sentence' column
df['processed_text'] = df['sentence'].apply(preprocess_text)

#display the first few rows of the dataframe with the new processed text
df.head()


,sentence,entities,processed_text
0,زار البابا فرنسيس مدينة أبوظبي لتعزيز الحوار ب...,"[(البابا فرنسيس, Person), (مدينة أبوظبي, Locat...",زار بابا فرنسيس مدينة وظبي تعزيز حوار بين ديان
1,أعلن نادي برشلونة عن انتقال اللاعب البرتغالي ك...,"[(نادي برشلونة, Organization), (اللاعب البرتغا...",علن نادي برشلونه عن انتقال لاعب برتغالي كريستي...
2,افتتحت شركة سامسونج معرضاً جديداً في مدينة دبي.,"[(شركة سامسونج, Organization), (مدينة دبي, Loc...",افتتح شركة سامسونج معرض جديد في مدينة دبي
3,زار الرئيس الفرنسي إيمانويل ماكرون لبنان بعد ا...,"[(الرئيس الفرنسي, Position), (إيمانويل ماكرون,...",زار ريس فرنسي يمانويل ماكر لبنان بعد انفجار مر...
4,ألقى الشاعر نزار قباني قصيدة في مهرجان قرطاج ا...,"[(الشاعر نزار قباني, Person), (مهرجان قرطاج ال...",لقى شاعر نزار قبان قصيدة في مهرجان قرطاج دولي


In [7]:
#save the new data into a csv file
df.to_csv('processed_arabic_ner_corpus_new.csv', index=False,encoding='utf-8-sig' )

The Arabic text undergone three main preprocessing processes in step 2:

It was cleaned using regular expressions to eliminate non-Arabic letters and special characters;

It was tokenized into individual words;

and finally, each token was reduced to its root form using the Farasa Stemmer.

Each phrase in the processed text is then converted to its stemmed form and added to a new column in the DataFrame.

Simplified versions of the original sentences with words reduced to their stems are displayed in the processed text.


***Step3: Develop Your Own NER (Rule-Based)***

In [8]:
import re

def own_rule_based_ner(tokens):
    #lets define lists of keywords for each entity type
    person_titles = ['البابا', 'الرئيس', 'الشاعر', 'إيمانويل', 'كريستيانو', 'نزار']
    location_keywords = ['مدينة', 'العاصمة', 'دولة', 'لبنان', 'مصر', 'الهند', 'كوبنهاغن', 'أبوظبي', 'دبي', 'باريس', 'بيروت']
    organization_keywords = ['شركة', 'جامعة', 'نادي', 'منظمة', 'وزارة', 'برشلونة', 'سامسونج', 'يونيسف', 'هارفارد', 'القاهرة']
    known_organizations = ['نادي برشلونة', 'شركة سامسونج', 'جامعة القاهرة']  #lets add multi-word organizations

    entities = []

    #iterate through tokens and identify entities
    i = 0
    while i < len(tokens):
        token = tokens[i]
        found_entity = False

        #check for known multi-word organizations first
        for org in known_organizations:
            org_tokens = org.split()
            if tokens[i:i + len(org_tokens)] == org_tokens:
                entities.extend([(t, 'Organization') for t in org_tokens])
                i += len(org_tokens)
                found_entity = True
                break

        if found_entity:
            continue

        #check for person titles
        if any(title in token for title in person_titles):
            entities.append((token, 'Person'))
        #check for keywords indicating an organization
        elif any(token.startswith(keyword) for keyword in organization_keywords):
            entities.append((token, 'Organization'))
        #check for location indicators
        elif token in location_keywords:
            entities.append((token, 'Location'))
        else:
            entities.append((token, 'O'))

        i += 1  #move to the next token

    return entities

#apply the rule-based NER to tokenized sentences
df['tokens'] = df['sentence'].apply(lambda x: x.split())  #tokenize sentences
df['predicted_entities'] = df['tokens'].apply(own_rule_based_ner)

#align labels with entities
def align_labels_with_entities(tokens, entities):
    labels = ['O'] * len(tokens)  #initialize labels as 'O'

    for i, (token, label) in enumerate(entities):
        labels[i] = label

    return labels

#create labels based on the predictions
df['labels'] = df.apply(lambda row: align_labels_with_entities(row['tokens'], row['predicted_entities']), axis=1)

#display the DataFrame with predictions and labels
print(df[['tokens', 'predicted_entities', 'labels']].head())


                                              tokens  \
0  [زار, البابا, فرنسيس, مدينة, أبوظبي, لتعزيز, ا...   
1  [أعلن, نادي, برشلونة, عن, انتقال, اللاعب, البر...   
2  [افتتحت, شركة, سامسونج, معرضاً, جديداً, في, مد...   
3  [زار, الرئيس, الفرنسي, إيمانويل, ماكرون, لبنان...   
4  [ألقى, الشاعر, نزار, قباني, قصيدة, في, مهرجان,...   

                                  predicted_entities  \
0  [(زار, O), (البابا, Person), (فرنسيس, O), (مدي...   
1  [(أعلن, O), (نادي, Organization), (برشلونة, Or...   
2  [(افتتحت, O), (شركة, Organization), (سامسونج, ...   
3  [(زار, O), (الرئيس, Person), (الفرنسي, O), (إي...   
4  [(ألقى, O), (الشاعر, Person), (نزار, Person), ...   

                                              labels  
0     [O, Person, O, Location, Location, O, O, O, O]  
1  [O, Organization, Organization, O, O, O, O, Pe...  
2  [O, Organization, Organization, O, O, O, Locat...  
3    [O, Person, O, Person, O, Location, O, O, O, O]  
4              [O, Person, Person, O, O, O, O, O, O

Step 3: The tokenized text is iterated through by the rule-based NER function, which uses predefined keywords for persons, locations, and organizations to identify entities.

The code creates a structured representation for additional assessment by aligning the labels with the tokens after predicting the entities.

Results:

The findings demonstrate the model's capacity to recognize individuals and organizations by displaying the tokenized sentences with the expected entities and the labels that correspond to them.

Although certain entities were accurately categorized, a large number of tokens are still marked as "O," suggesting that named entity recognition still needs work.



***Step 4: Evaluate the Results***

In [9]:
from sklearn.metrics import classification_report

#lets create a function to align true labels with all tokens in the sentence
def align_true_labels_with_tokens(tokens, entities):
    labels = ['O'] * len(tokens)  #initialize all tokens with 'O'

    for entity, label in entities:
        entity_tokens = entity.split()
        for i in range(len(tokens) - len(entity_tokens) + 1):
            if tokens[i:i + len(entity_tokens)] == entity_tokens:
                for j in range(len(entity_tokens)):
                    labels[i + j] = label
                break
    return labels

#create aligned true labels for each sentence
df['aligned_true_labels'] = df.apply(lambda row: align_true_labels_with_tokens(row['tokens'], row['entities']), axis=1)

#flatten a list of lists into a single list
def flatten_lists(lists):
    return [item for sublist in lists for item in sublist]

#flatten the lists of aligned true labels and predicted labels
true_labels = flatten_lists(df['aligned_true_labels'])
predicted_labels = flatten_lists(df['labels'])

#generate the classification report
report = classification_report(true_labels, predicted_labels, labels=['Person', 'Organization', 'Location', 'O'], zero_division=0)

print(report)


              precision    recall  f1-score   support

      Person       0.83      0.56      0.67         9
Organization       1.00      0.73      0.84        11
    Location       0.86      0.60      0.71        10
           O       0.75      0.98      0.85        56

   micro avg       0.79      0.86      0.82        86
   macro avg       0.86      0.72      0.77        86
weighted avg       0.81      0.86      0.81        86



In step 4, we used measures like precision, recall, and F1-score to assess how well the rule-based NER model performed. This stage gives us an understanding of how well the model matches the ground truth in terms of entity identification.

Results:

The model exhibits low recall for "Person" (0.56) and "Location" (0.60), but good precision, particularly for "Organization" entities (1.00).

The model does very well overall, but there is still space for improvement, especially in detecting "Person" and "Location" entities, as indicated by the weighted F1-score of 0.81.



